# Spatial averaging/ Coarse graining

Here we do a coarse graining on the time averaged fields. 

In [1]:
# load modules
import xarray as xr
import numpy  as np
from xgcm import Grid
%matplotlib inline

In [16]:
# Load time averages 
Model_Tav = xr.open_dataset('./data/Model_Tav_3D.nc')
Trac_Tav  = xr.open_dataset('./data/Trac_Tav_3D.nc')

In [17]:
Trac_Tav

<xarray.Dataset>
Dimensions:   (XC: 140, XG: 140, YC: 240, YG: 240, Z: 40, Zl: 40, Zp1: 41, Zu: 40)
Coordinates:
  * XC        (XC) float32 2500.0 7500.0 12500.0 ... 687500.0 692500.0 697500.0
  * YC        (YC) float32 2500.0 7500.0 12500.0 ... 1192500.0 1197500.0
  * XG        (XG) float32 0.0 5000.0 10000.0 ... 685000.0 690000.0 695000.0
  * YG        (YG) float32 0.0 5000.0 10000.0 ... 1185000.0 1190000.0 1195000.0
  * Z         (Z) float32 -2.5 -7.5 -12.5 -18.0 ... -1374.0 -1426.0 -1478.0
  * Zp1       (Zp1) float32 0.0 -5.0 -10.0 -15.0 ... -1400.0 -1452.0 -1504.0
  * Zu        (Zu) float32 -5.0 -10.0 -15.0 -21.0 ... -1400.0 -1452.0 -1504.0
  * Zl        (Zl) float32 0.0 -5.0 -10.0 -15.0 ... -1348.0 -1400.0 -1452.0
    rA        (YC, XC) float32 ...
    dxG       (YG, XC) float32 ...
    dyG       (YC, XG) float32 ...
    Depth     (YC, XC) float32 ...
    rAz       (YG, XG) float32 ...
    dxC       (YC, XG) float32 ...
    dyC       (YG, XC) float32 ...
    rAw       (YC, XG) fl

In [18]:
grid = Grid(Model_Tav, periodic='X')

In [19]:
list_tracers = ['TRAC01', 'TRAC02','TRAC03','TRAC04','TRAC05',
                'TRAC06','TRAC07','TRAC08','TRAC09']

list_tracers2 = ['Tr01', 'Tr02','Tr03','Tr04','Tr05',
                'Tr06','Tr07','Tr08','Tr09']

vels = {'uVeltave':'U', 'vVeltave':'V', 'wVeltave':'W'}

In [20]:
Lcoarse = 50e3 # needs to be a multiple of the domain size.
idmaxx   = int( (Model_Tav.dxC[0,0].values * len(Model_Tav.XC))/Lcoarse) # numper of grid points to average over
nrepsx   = int( len(Model_Tav.XC)/idmaxx )
idmaxy   = int( (Model_Tav.dyC[0,0].values * len(Model_Tav.YC))/Lcoarse) # numper of grid points to average over
nrepsy   = int( len(Model_Tav.YC)/idmaxy )

In [21]:
# Coarse grain velocities 

YCicoarse = np.repeat(range(idmaxy),nrepsy)
XCicoarse = np.repeat(range(idmaxx),nrepsx)

# Interpolate mean velocities to all be on same point.# Interp 
Model_Tav['WmeanC'] = grid.interp(Model_Tav.wVeltave, 'Z', boundary='extend')
Model_Tav['VmeanC'] = grid.interp(Model_Tav.vVeltave, 'Y', boundary='extend')
Model_Tav['UmeanC'] = grid.interp(Model_Tav.uVeltave, 'X')

Model_Tav.coords['XCicoarse'] = ('XC', XCicoarse)
Model_Tav.coords['YCicoarse'] = ('YC', YCicoarse)


YCcoarse = ('YCicoarse', Model_Tav.YC.groupby('YCicoarse').mean('YC'))
XCcoarse = ('XCicoarse', Model_Tav.XC.groupby('XCicoarse').mean('XC'))

# coarse grain mean velocities

Vels_coarse = xr.Dataset()

for i in vels:
    temp = (Model_Tav[vels[i]+'meanC'].groupby('YCicoarse').mean('YC')).groupby('XCicoarse').mean('XC')
    
    temp_array = xr.DataArray(temp.data, coords = [temp.Z, YCcoarse, XCcoarse], 
                             dims = ['Z', 'YCcoarse', 'XCcoarse'])
    
    Vels_coarse[vels[i]] = temp_array
    
hFacCcoarse = ((Model_Tav.hFacC.groupby('YCicoarse').mean('YC')).groupby('XCicoarse').mean('XC'))

Vels_coarse.coords['hFacCcoarse'] = (['Z', 'YCcoarse', 'XCcoarse'], hFacCcoarse)

In [22]:
# Move eddy fluxes to center point
for key in list_tracers2: 
    Trac_Tav['DFrI'+key] = grid.interp(Trac_Tav['DFrI'+key],'Z', boundary='extend')
    
for key in list_tracers: 
        Trac_Tav['V'+key] = grid.interp(Trac_Tav['V'+key], 'Y', boundary='extend')
        Trac_Tav['U'+key] = grid.interp(Trac_Tav['U'+key], 'X')
        
# create coarse grained version of fluxes and mean tracers# create 

Trac_Tav.coords['YCicoarse'] = ('YC', YCicoarse)
Trac_Tav.coords['XCicoarse'] = ('XC', XCicoarse)

YCcoarse = ('YCicoarse', Trac_Tav.YC.groupby('YCicoarse').mean('YC'))
XCcoarse = ('XCicoarse', Trac_Tav.XC.groupby('XCicoarse').mean('XC'))

In [23]:
Trac_Tav

<xarray.Dataset>
Dimensions:    (XC: 140, XG: 140, YC: 240, YG: 240, Z: 40, Zl: 40, Zp1: 41, Zu: 40)
Coordinates:
  * XC         (XC) float32 2500.0 7500.0 12500.0 ... 687500.0 692500.0 697500.0
  * YC         (YC) float32 2500.0 7500.0 12500.0 ... 1192500.0 1197500.0
  * XG         (XG) float32 0.0 5000.0 10000.0 ... 685000.0 690000.0 695000.0
  * YG         (YG) float32 0.0 5000.0 10000.0 ... 1185000.0 1190000.0 1195000.0
  * Z          (Z) float32 -2.5 -7.5 -12.5 -18.0 ... -1374.0 -1426.0 -1478.0
  * Zp1        (Zp1) float32 0.0 -5.0 -10.0 -15.0 ... -1400.0 -1452.0 -1504.0
  * Zu         (Zu) float32 -5.0 -10.0 -15.0 -21.0 ... -1400.0 -1452.0 -1504.0
  * Zl         (Zl) float32 0.0 -5.0 -10.0 -15.0 ... -1348.0 -1400.0 -1452.0
    rA         (YC, XC) float32 ...
    dxG        (YG, XC) float32 ...
    dyG        (YC, XG) float32 ...
    Depth      (YC, XC) float32 ...
    rAz        (YG, XG) float32 ...
    dxC        (YC, XG) float32 ...
    dyC        (YG, XC) float32 ...
    rAw  

In [24]:
Trac_coarse = xr.Dataset()

for i in Trac_Tav.data_vars:
    temp_var = (Trac_Tav[i].groupby('YCicoarse').mean('YC')).groupby('XCicoarse').mean('XC')
    temp = xr.DataArray(temp_var.data, coords = [temp_var.Z, YCcoarse, XCcoarse], 
                             dims = ['Z', 'YCcoarse', 'XCcoarse'] )
    Trac_coarse[i] = temp

In [25]:
# flux datasets 

for i in list_tracers: 
    Trac_coarse['Up'+i+'p'] = Trac_coarse['U'+i] - Vels_coarse['U'] * Trac_coarse[i]    
    Trac_coarse['Vp'+i+'p'] = Trac_coarse['V'+i] - Vels_coarse['V'] * Trac_coarse[i]
    Trac_coarse['Wp'+i+'p'] = Trac_coarse['W'+i] - Vels_coarse['W'] * Trac_coarse[i]

In [26]:
Trac_coarse.coords['drF'] = Trac_Tav.drF
Trac_coarse.coords['hFacCcoarse'] = (['Z', 'YCcoarse', 'XCcoarse'], hFacCcoarse)

In [27]:
# Calculate gradients 

# Z grads (don't need staggered grid)
for i in list_tracers:
    Trac_coarse[i+'_Z'] = -grid.diff(grid.interp(Trac_coarse[i], 'Z', boundary='extend'),
                                    'Z', boundary='extend')/Trac_coarse.drF

In [28]:
# To calculate the tracer gradients we need a staggered coordinate

# make staggered coordinates
tempy = np.repeat(idmaxy, nrepsy/2)
YSicoarse = np.concatenate([YCicoarse[int(nrepsy/2):], tempy])

tempx = np.repeat(idmaxx-1, nrepsx/2)
XSicoarse = np.concatenate([tempx, XCicoarse[:-int(nrepsx/2)]])

trac_stag = xr.Dataset()

for i in list_tracers: 
    trac_stag[i] = Trac_Tav[i]
    
trac_stag.coords['XSicoarse'] = ('XC', XSicoarse)
trac_stag.coords['YSicoarse'] = ('YC', YSicoarse)

In [29]:
# Y gradient
# only average in Y direction on staggered points to take Y average. 
# average in X direction is on center points
tracsav_stag = xr.Dataset()
for i in list_tracers:
    tracsav_stag[i] = (trac_stag[i].groupby('YSicoarse').mean('YC')).groupby('XCicoarse').mean('XC')
    
for i in list_tracers:
    temp = tracsav_stag[i].diff('YSicoarse')/Lcoarse
    # dy on the end points in Y direction are not full size
    temp[:,0,:] = temp[:,0,:] * 4/3
    temp[:,-1,:] = temp[:,-1,:] * 4/3 
    
    temp = xr.DataArray(temp.data, coords=[Trac_coarse.Z, Trac_coarse.YCcoarse, Trac_coarse.XCcoarse, ], 
                       dims=['Z', 'YCcoarse', 'XCcoarse'])
    Trac_coarse[i+'_Y'] = temp
    

In [30]:
# X grads 
# only averate in X direction on center points
tracsav_stag = xr.Dataset()
for i in list_tracers:
    tracsav_stag[i] = (trac_stag[i].groupby('YCicoarse').mean('YC')).groupby('XSicoarse').mean('XC')

for i in list_tracers:
    
    temp = (tracsav_stag[i].data - np.roll(tracsav_stag[i].data,1,axis=2))/Lcoarse
    
    temp = xr.DataArray(temp.data, coords=[Trac_coarse.Z, Trac_coarse.YCcoarse, Trac_coarse.XCcoarse, ], 
                       dims=['Z', 'YCcoarse', 'XCcoarse'])
        
    Trac_coarse[i+'_X'] = temp    

In [31]:
Trac_coarse.to_netcdf('data/Trac_coarse_'+str(int(Lcoarse//1e3))+'km_3D.nc')
Vels_coarse.to_netcdf('data/Vels_coarse_'+str(int(Lcoarse//1e3))+'km_3D.nc')

## Spatial Averaging the Temperature Fields

In [32]:
# Temperature fields

Tmean = Model_Tav.Ttave
Tmean.coords['YCicoarse'] = ('YC', YCicoarse)
Tmean.coords['XCicoarse'] = ('XC', XCicoarse)

T_coarse = xr.Dataset()

YCcoarse = ('YCicoarse', Tmean.YC.groupby('YCicoarse').mean('YC'))
XCcoarse = ('XCicoarse', Tmean.XC.groupby('XCicoarse').mean('XC'))
# coarse grain mean velocities


temp = (Tmean.groupby('YCicoarse').mean('YC')).groupby('XCicoarse').mean('XC')
    
temp_array = xr.DataArray(temp.data, coords = [temp.Z, YCcoarse, XCcoarse], 
                            dims = ['Z', 'YCcoarse', 'XCcoarse'])
    
T_coarse['T'] = temp_array

In [33]:
T_coarse.coords['hFacCcoarse'] = (['Z', 'YCcoarse', 'XCcoarse'], hFacCcoarse)

In [34]:
# Z grads
T_coarse['T_Z'] = -grid.diff(grid.interp(T_coarse['T'], 'Z', boundary='extend'),
                                    'Z', boundary='extend')/Tmean.drF

In [35]:
Tstag = Tmean
Tstag.coords['YSicoarse'] = ('YC', YSicoarse)
Tstag.coords['XSicoarse'] = ('XC', XSicoarse)

# Y grads 
Tstag_sav = (Tstag.groupby('YSicoarse').mean('YC')).groupby('XCicoarse').mean('XC')

temp = Tstag_sav.diff('YSicoarse')/Lcoarse
temp[:,0,:] = temp[:,0,:] * 4/3
temp[:,-1,:] = temp[:,-1,:] * 4/3   
    
temp = xr.DataArray(temp.data, dims=['Z', 'YCcoarse', 'XCcoarse'],
                    coords=[ T_coarse.Z, T_coarse.YCcoarse,T_coarse.XCcoarse])
    
T_coarse['T_Y'] = temp

In [36]:
# X grads 
# only averate in X direction on center points
Tstag_sav = (Tstag.groupby('YCicoarse').mean('YC')).groupby('XSicoarse').mean('XC')

temp = (Tstag_sav.data - np.roll(Tstag_sav.data,1,axis=2))/Lcoarse
temp = xr.DataArray(temp.data, dims=['Z', 'YCcoarse', 'XCcoarse'],
                    coords=[T_coarse.Z ,T_coarse.YCcoarse, T_coarse.XCcoarse])
        
T_coarse['T_X'] = temp

In [37]:
Model_Tav

<xarray.Dataset>
Dimensions:    (XC: 140, XG: 140, YC: 240, YG: 240, Z: 40, Zl: 40, Zp1: 41, Zu: 40)
Coordinates:
  * XC         (XC) float32 2500.0 7500.0 12500.0 ... 687500.0 692500.0 697500.0
  * YC         (YC) float32 2500.0 7500.0 12500.0 ... 1192500.0 1197500.0
  * XG         (XG) float32 0.0 5000.0 10000.0 ... 685000.0 690000.0 695000.0
  * YG         (YG) float32 0.0 5000.0 10000.0 ... 1185000.0 1190000.0 1195000.0
  * Z          (Z) float32 -2.5 -7.5 -12.5 -18.0 ... -1374.0 -1426.0 -1478.0
  * Zp1        (Zp1) float32 0.0 -5.0 -10.0 -15.0 ... -1400.0 -1452.0 -1504.0
  * Zu         (Zu) float32 -5.0 -10.0 -15.0 -21.0 ... -1400.0 -1452.0 -1504.0
  * Zl         (Zl) float32 0.0 -5.0 -10.0 -15.0 ... -1348.0 -1400.0 -1452.0
    rA         (YC, XC) float32 ...
    dxG        (YG, XC) float32 ...
    dyG        (YC, XG) float32 ...
    Depth      (YC, XC) float32 ...
    rAz        (YG, XG) float32 ...
    dxC        (YC, XG) float32 ...
    dyC        (YG, XC) float32 ...
    rAw  

In [38]:
Model_Tav['UTtave'] = grid.interp(Model_Tav.UTtave, 'X')
Model_Tav['VTtave'] = grid.interp(Model_Tav.VTtave, 'Y', boundary='extend')
Model_Tav['WTtave'] = grid.interp(Model_Tav.WTtave, 'Z', boundary='extend')

Model_Tav.coords['YCicoarse'] = ('YC', YCicoarse)
Model_Tav.coords['XCicoarse'] = ('XC', XCicoarse)

for i in ['UT','VT','WT','TT']: 
    temp = (Model_Tav[i+'tave'].groupby('YCicoarse').mean('YC')).groupby('XCicoarse').mean('XC') 
    temp = xr.DataArray(temp.data, dims=['Z', 'YCcoarse', 'XCcoarse'],
                    coords=[T_coarse.Z ,T_coarse.YCcoarse, T_coarse.XCcoarse])
    T_coarse[i] = temp
    
T_coarse['UpTp'] = T_coarse['UT'] - Vels_coarse['U'] *T_coarse['T']
T_coarse['VpTp'] = T_coarse['VT'] - Vels_coarse['V'] *T_coarse['T']
T_coarse['WpTp'] = T_coarse['WT'] - Vels_coarse['W'] *T_coarse['T']
T_coarse['TpTp'] = T_coarse['TT'] - T_coarse['T']**2

In [39]:
T_coarse

<xarray.Dataset>
Dimensions:      (XCcoarse: 14, YCcoarse: 24, Z: 40)
Coordinates:
  * Z            (Z) float32 -2.5 -7.5 -12.5 -18.0 ... -1374.0 -1426.0 -1478.0
  * YCcoarse     (YCcoarse) float32 25000.0 75000.0 ... 1125000.0 1175000.0
  * XCcoarse     (XCcoarse) float32 25000.0 75000.0 ... 625000.0 675000.0
    hFacCcoarse  (Z, YCcoarse, XCcoarse) float32 0.9 0.9 ... 0.9230769 0.9230769
    drF          (Z) float32 ...
    PHrefC       (Z) float32 ...
Data variables:
    T            (Z, YCcoarse, XCcoarse) float32 1.3140967 ... 0.92313766
    T_Z          (Z, YCcoarse, XCcoarse) float32 -3.118515e-05 ... -0.0
    T_Y          (Z, YCcoarse, XCcoarse) float32 9.538218e-06 ... 4.2023657e-08
    T_X          (Z, YCcoarse, XCcoarse) float32 -7.857323e-08 ... 3.6406518e-09
    UT           (Z, YCcoarse, XCcoarse) float32 0.022464532 ... 0.023961294
    VT           (Z, YCcoarse, XCcoarse) float32 0.0009504386 ... -0.0005595804
    WT           (Z, YCcoarse, XCcoarse) float32 2.540608e-07

In [41]:
T_coarse.to_netcdf('data/T_coarse_'+str(int(Lcoarse//1e3))+'km_3D.nc')